In [1]:
import pandas as pd
from lifetimes import BetaGeoBetaBinomFitter
from lifetimes.datasets import load_donations

In [19]:
discrete_noncontract_df = load_donations()

periods = 6
bgbb = BetaGeoBetaBinomFitter().fit(discrete_noncontract_df['frequency'].values,
                             discrete_noncontract_df['recency'].values,
                             discrete_noncontract_df['periods'].values,
                             discrete_noncontract_df['weights'].values)
bgbb

<lifetimes.BetaGeoBetaBinomFitter: fitted with 22 subjects, alpha: 1.20, beta: 0.75, delta: 2.78, gamma: 0.66>

In [16]:
method_list = [method for method in dir(BetaGeoBetaBinomFitter) if not method.startswith('_')]
print(method_list)



['conditional_expected_number_of_purchases_up_to_time', 'conditional_probability_alive', 'expected_number_of_transactions_in_first_n_periods', 'fit', 'load_model', 'save_model', 'summary']


In [30]:
# equation 13 in paper
bgbb.conditional_expected_number_of_purchases_up_to_time(5,
    discrete_noncontract_df['frequency'],
    discrete_noncontract_df['recency'],
    discrete_noncontract_df['periods'])

0     0.072863
1     0.085696
2     0.314238
3     0.593853
4     0.839396
5     1.021689
6     1.147885
7     0.119121
8     0.536111
9     1.057604
10    1.443042
11    1.668817
12    0.223595
13    1.034572
14    1.804703
15    2.189749
16    0.583192
17    2.030024
18    2.710681
19    1.812942
20    3.231612
21    3.752544
dtype: float64

In [25]:
# equation 11 in paper
bgbb.conditional_probability_alive(10,
    discrete_noncontract_df['frequency'],
    discrete_noncontract_df['recency'],
    discrete_noncontract_df['periods'])

0     0.070072
1     0.045012
2     0.165056
3     0.311927
4     0.440900
5     0.536651
6     0.602936
7     0.043038
8     0.193695
9     0.382108
10    0.521365
11    0.602936
12    0.061566
13    0.284864
14    0.496916
15    0.602936
16    0.129719
17    0.451538
18    0.602936
19    0.338249
20    0.602936
21    0.602936
dtype: float64

In [63]:
# TODO: write and test (8) as a replacement. Compare against just aggregating means across the exploded DF 
# TODO: Can the arviz functions in the BetaGeoBetaBinom distribution block preclude the need for this?
# TODO: Replace this with (9) or (10) in a future PR, since that expression can predict interval ranges

# equation 7 in paper, but that's for probabilities. should it be 8 for predicting mean n?
# yeah, this function should be renamed for clarity. 
# it distributes customers in the dataset across n transaction opportunies
# it works better as an evaluation function, since it assumes a fixed customer population size
# if n > n_periods, it will keep right on predicting. This may be a bug
bgbb.expected_number_of_transactions_in_first_n_periods(n=50)

,model
frequency,
0,3195.925987
1,1560.549020
2,964.135361
3,668.795916
4,497.960966
5,389.113685
6,314.983874


In [45]:
discrete_noncontract_df

,frequency,recency,periods,weights
0,0,0,6,3464
1,1,1,6,1091
2,1,2,6,277
3,1,3,6,129
4,1,4,6,78
5,1,5,6,119
6,1,6,6,129
7,2,2,6,613
8,2,3,6,255
9,2,4,6,155
